# The Depth-AI Pipeline

## Capturing images from the OAK-D-Lite camera

(ref: https://learnopencv.com/introduction-to-opencv-ai-kit-and-depthai/)

In [ ]:
import depthai as dai
import matplotlib.pyplot as plt

import numpy as np
import cv2

import time

In [ ]:
# Function to get the frame from the queue, transfer it 
# to the host and convert it to a numpy array
def getFrame(queue):
    # Get frame from queue
    frame = queue.get()
    # Convert frame to OpenCV format and return
    return frame.getCvFrame()

In [ ]:
# Function to select mono camera
def getMonoCamera(pipeline, isLeft):
    # Configure mono camera
    mono = pipeline.createMonoCamera()
    
    # Set camera resolution
    mono.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)
    if isLeft:
        mono.setBoardSocket(dai.CameraBoardSocket.LEFT)
    else:
        mono.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    return mono

def getRGBCamera(pipeline):
    rgb_cam = pipeline.createColorCamera()
    # rgb_cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_4000X3000)
    rgb_cam.setBoardSocket(dai.CameraBoardSocket.RGB)
    return rgb_cam

In [ ]:
pipeline = dai.Pipeline()

# Set up cameras
monoLeft = getMonoCamera(pipeline, isLeft=True)
monoRight = getMonoCamera(pipeline, isLeft=False)
centerRGB = getRGBCamera(pipeline)

# Set output XLink for cameras
xoutLeft = pipeline.createXLinkOut()
xoutLeft.setStreamName('left')

xoutRight = pipeline.createXLinkOut()
xoutRight.setStreamName('right')

xoutRGB = pipeline.createXLinkOut()
xoutRGB.setStreamName('rgb')

# Attach cameras to output XLink
monoLeft.out.link(xoutLeft.input)
monoRight.out.link(xoutRight.input)
centerRGB.video.link(xoutRGB.input)

In [ ]:
gap = 20
mono_res = (480,640)

# Transfer pipeline to device
with dai.Device(pipeline) as device:
    # Get output queues
    leftQueue = device.getOutputQueue(name='left', maxSize=1)
    rightQueue = device.getOutputQueue(name='right', maxSize=1)
    rgbQueue = device.getOutputQueue(name='rgb', maxSize=1)
    
    # Set display window name
    cv2.namedWindow('Stereo Pair', cv2.WINDOW_NORMAL)
    cv2.namedWindow('RGB Camera', cv2.WINDOW_NORMAL)
    
    stereo_canvas = np.zeros([mono_res[0],mono_res[1]*2+gap])
    
    # main loop
    while True:
        # Get frames
        leftFrame = getFrame(leftQueue)
        rightFrame = getFrame(rightQueue)
        rgbFrame = getFrame(rgbQueue)
        
        stereo_canvas[:,:mono_res[1]] = leftFrame / 255.
        stereo_canvas[:,mono_res[1]+gap:] = rightFrame / 255.
        
        cv2.imshow('Stereo Pair', stereo_canvas)
        cv2.imshow('RGB Camera', rgbFrame)
        
        key = cv2.waitKey(1)
        if key == ord('q'):
            print('QUIT')
            break

In [ ]:
leftFrame